## `Assignment 1`
### `MNIST dataset`

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

In [2]:
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

In [3]:
mnist_folder = 'data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


In [4]:
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

In [5]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

In [6]:
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.shuffle(5000)
test_data = test_data.batch(batch_size)

In [7]:
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

In [8]:
train_init = iterator.make_initializer(train_data)   # initializer for train_data
test_init  = iterator.make_initializer(test_data)  # initializer for test_data

In [9]:
w = tf.get_variable('weights',shape=(784,10),initializer=tf.random_normal_initializer(0.0,0.1))
b = tf.get_variable('bias',shape=(1,10),initializer=tf.zeros_initializer())

In [10]:
logits = tf.add(tf.matmul(img,w),b,name='logits')

In [11]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss')

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate).compute_gradients(loss)

In [13]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./tmp/graph/logreg', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
 
    # train the model n_epochs times
    for i in range(n_epochs):
        sess.run(train_init)   # drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)   # drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.7922738654668942
Average loss epoch 1: 0.7922655065392339
Average loss epoch 2: 0.7922409782576006
Average loss epoch 3: 0.7922646009644797
Average loss epoch 4: 0.7922315951003585
Average loss epoch 5: 0.7922874029292617
Average loss epoch 6: 0.7921543776988983
Average loss epoch 7: 0.7921709329582924
Average loss epoch 8: 0.7922855477000392
Average loss epoch 9: 0.7922679181708846
Average loss epoch 10: 0.7921707881051441
Average loss epoch 11: 0.7922647659168687
Average loss epoch 12: 0.792207630429157
Average loss epoch 13: 0.792213356633519
Average loss epoch 14: 0.7922837779965511
Average loss epoch 15: 0.7922703047131383
Average loss epoch 16: 0.792274767576262
Average loss epoch 17: 0.7922056023464646
Average loss epoch 18: 0.7922477522561716
Average loss epoch 19: 0.7922778911368792
Average loss epoch 20: 0.7921714167262233
Average loss epoch 21: 0.7922407294428626
Average loss epoch 22: 0.7922627907852794
Average loss epoch 23: 0.7921875047129254
Avera